In [ ]:
import numpy as np
from scipy import interpolate
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Timestamp


def get_dates_NB_ent(x):
    # Select the df values between start and ending datetime. 
    k = frame_NB_ent[(frame_NB_ent['eventdatetime']>=x['start'])&(frame_NB_ent['eventdatetime']<x['end'])]
    
    try: 
        return k['rid'].values[0], x['start'], k['value'].sum()
    except IndexError:
        return 0, x['start'], 0
    else:
        return -1, -1, -1
    
def in_range(x, i):
    k = Data[(Data['value']>=x[0])&(Data['value']<=x[len(x) - 1])]
    k['value'] = i
    return k

#read from data.csv    
frame = pd.read_csv('data.csv', header=0, sep=';')
frame['building'], frame['terminal'] = frame['event_point'].str.split('-', 1).str
frame['eventdatetime'] =  pd.to_datetime(frame['eventdatetime'], format = '%d.%m.%Y %H:%M')
frame['value'] = 1
del frame['event_point']

#New Building entrance frame
frame_NB_ent = frame[(frame['building'] == 'NB')&(frame['event_name'] == 'entrance')]
del frame_NB_ent['building']
del frame_NB_ent['event_name']

#make 5 min tiks
n_0 = 1483315500+2591700
n_end = 1494201300
Data = pd.DataFrame(columns = ['rid', 'time', 'value'])
n = n_0
end_date = pd.Timestamp(n, unit='s')
while(n < n_end):
    start_date = end_date 
    end_date = pd.Timestamp(n + 300, unit='s') 
    ndf = pd.DataFrame({'start':start_date,'end':end_date}, index=[0])
    n = n + 300
    ndf[['rid','time', 'value']] = ndf.apply(lambda x : get_dates_NB_ent(x),1).apply(pd.Series)
    new_line = {'rid':ndf['rid'].values[0], 'time':ndf['time'].values[0], 'value':ndf['value'].values[0]}
    Data = Data.append(new_line, ignore_index = True)


In [ ]:
#save
Data.to_csv('5_min_all_months_NB_ent.csv', sep=';', header=True, index=False)

In [ ]:
#discretization and sort by time, save
value_area = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15,
              17, 19, 24, 29, 34, 39, 44, 49, 54, 59, 69, 79, 89, 99, 119, 139, 159, 179]
value_area_bits = dict()
for x in range(len(value_area) - 1):
    value_area_bits[x] = range(value_area[x], value_area[x+1])
value_area_bits[32] = range(value_area[32], 260)
Data_discr = pd.DataFrame(columns = ['rid', 'time', 'value'])
Data_discr = pd.concat([in_range(value_area_bits[i], i) for i in range(len(value_area))], ignore_index=True)
Data_discr = Data_discr.sort_values(by='time').reset_index()
del Data_discr['index']
Data_discr.to_csv('Discr_sort_all.csv', sep=';', header=True, index=False)

In [ ]:
# the histogram of the data
n = 27483
x = Data_discr[['value']]
%pylab
plt.figure(figsize(8,6))

plt.hist(x['value'], bins =  np.arange(0, n, 1),  facecolor='red')
plt.xlabel('ticks')
plt.ylabel('Values')
plt.title('Values')
plt.xlim(0, 35)
plt.ylim(0, 10000)
plt.grid(True)
plt.show()

In [ ]:
# the plot of the data
%pylab
plt.figure(figsize(10,8))
a = plt.axes([0.1, 0.1, 0.8, 0.8],)
plt.plot(np.arange(0, n, 1), x)
plt.title('NB_ent')
plt.xlim(0, 27500)
plt.xticks([])
plt.yticks([])
plt.xlabel('time')
plt.ylabel('Values')
plt.grid(True)
plt.show()

In [ ]:
#make patterns


def make_patterns():
    
    Data = pd.read_csv('Discr_sort_all.csv', header=0, sep=';')
    Data.time = Data.time.apply(pd.to_datetime)
    Data['weekday'] = Data['time'].apply(pd.Timestamp.weekday)
    
    pat_columns = list(range(-1,13))
    pat_columns[0] = 'weekday'
    pat_columns[13] = 'time'
    pattern = pd.DataFrame(columns = pat_columns)
    
    for num in range(27468):
        column = Data[(Data.index >= num)&(Data.index < num + 12)].value
        new_line_dict = {i:column[i+num] for i in range(12)}
        new_line_dict['time'] = Data.time[num]
        new_line_dict['weekday'] = Data.weekday[num]
        pattern = pattern.append(new_line_dict, ignore_index = True)
        
    pattern.to_csv('pattern.csv', sep=';', header=True, index=False)
    return pattern



In [ ]:
#repeat number(probability * N)
pattern = pd.read_csv('pattern.csv', header=0, sep=';')
del pattern['time']
pattern['flag'] = 0

new_pattern = pd.DataFrame(columns = pat_columns)
del new_pattern['time']
for num in range(27468):
    column = Data[(Data.index >= num)&(Data.index < num + 12)].value
    line = {i:column[i+num] for i in range(12)}
    line['weekday'] = Data.weekday[num]
    new_pattern = pattern[(pattern['0'] == line[0]) & (pattern['1'] == line[1]) & (pattern['2'] == line[2]) &
                                       (pattern['3'] == line[3]) &(pattern['4'] == line[4]) &(pattern['5'] == line[5]) &
                                       (pattern['6'] == line[6]) &(pattern['7'] == line[7]) &(pattern['8'] == line[8]) &
                                       (pattern['9'] == line[9]) &(pattern['10'] == line[10]) &(pattern['11'] == line[11]) &
                                       (pattern['weekday'] == line['weekday']) & (pattern['flag'] == 0)].copy(deep=True)

    value_of_str = len(new_pattern)
    for index, row in new_pattern.iterrows():
        pattern.at[index, 'prob'] = value_of_str
        pattern.at[index, 'flag'] = 1
        
pattern.to_csv('pattern_with_pr.csv', sep=';', header=True, index=False)

In [ ]:
pattern = pd.read_csv('pattern.csv', header=0, sep=';')
del pattern['time']
pattern['flag'] = 0

def make_pat_with_pr_without_rep_from_pat():

    Data = pd.read_csv('Discr_sort_all.csv', header=0, sep=';')
    Data.time = Data.time.apply(pd.to_datetime)
    Data['weekday'] = Data['time'].apply(pd.Timestamp.weekday)
    pat_columns = list(range(-1,13))
    pat_columns[0] = 'weekday'
    pat_columns[13] = 'time'

    pat_columns_pr = list(range(-1,14))
    pat_columns_pr[0] = 'weekday'
    pat_columns_pr[14] = 'prob'
    pat_columns_pr[13] = 'flag'
    
    new_pat_dict = dict()
    dict_str = []
    new_pattern = pd.DataFrame(columns = pat_columns)
    del new_pattern['time']
    for num in range(27468):
        column = Data[(Data.index >= num)&(Data.index < num + 12)].value
        line = {i:column[i+num] for i in range(12)}
        line['weekday'] = Data.weekday[num]
        new_pattern = pattern[(pattern['0'] == line[0]) & (pattern['1'] == line[1]) & (pattern['2'] == line[2]) &
                                           (pattern['3'] == line[3]) &(pattern['4'] == line[4]) &(pattern['5'] == line[5]) &
                                           (pattern['6'] == line[6]) &(pattern['7'] == line[7]) &(pattern['8'] == line[8]) &
                                           (pattern['9'] == line[9]) &(pattern['10'] == line[10]) &(pattern['11'] == line[11]) &
                                           (pattern['weekday'] == line['weekday']) & (pattern['flag'] == 0)].copy(deep=True)
        
        value_of_str = len(new_pattern)
        for index, row in new_pattern.iterrows():
            pattern.at[index, 'flag'] = 1
            new_pattern.at[index, 'prob'] = value_of_str

        if(new_pattern[:1].empty == False):
            dict_str.append(dict(zip(pat_columns_pr, np.array(new_pattern[:1]).reshape(15,).tolist())))
    new_pattern_end = pd.DataFrame(dict_str)
    new_pattern_end.to_csv('pat_with_pr_without_rep.csv', sep=';', header=True, index=False)
    
    return new_pattern_end
